In [151]:
import random
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from geopandas import GeoDataFrame
import xlrd
from random import *
import requests
from io import BytesIO

pd.set_option('display.max_columns', None)

In [152]:
gdf = gpd.read_file("./shp/SF_DOE_PREC_2022_07_18_pg.shp", encoding='utf-8')
gdf = gdf.to_crs('EPSG:4269')

In [153]:
url = 'https://www.sfelections.org/results/20250916/data/20250807/sov.xlsx'

In [154]:
response = requests.get(url)

In [155]:
dfTurnout = pd.read_excel(BytesIO(response.content), sheet_name='Sheet1', header=4, skipfooter=10)
dfA = pd.read_excel(BytesIO(response.content), sheet_name='Sheet2', header=3, skipfooter=8)

In [156]:
dfA

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,Yes\n,Unnamed: 7,No\n,Unnamed: 9,Total Votes,Unnamed: 11
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
4,Vote by Mail,2566.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
80,Vote by Mail,2383.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN


In [157]:
###
### TURNOUT
###

#trim and rename columns
dfTurnout = dfTurnout[['Precinct','Registered\nVoters','Voters Cast']]
dfTurnout.columns = ['precinct','registered_voters','votes_cast']

# shift precinct column

dfTurnout['precinct'] = dfTurnout['precinct'].shift(3)

#drop unnecessary ones
dfTurnout = dfTurnout.dropna()
dfTurnout = dfTurnout[dfTurnout["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfTurnout['precinct'] = dfTurnout['precinct'].str.replace('PCT ','').str.replace(' MB','')

#change data types
dfTurnout.registered_voters = dfTurnout.registered_voters.astype(int)
dfTurnout.votes_cast = dfTurnout.votes_cast.astype(int)

#create turnout column
dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1)
dfTurnout['turnout'] = dfTurnout['turnout'].fillna(0)

In [158]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout
5,9401,2566,0,0.0
9,9403,2247,0,0.0
13,9404,3243,0,0.0
17,9406,2415,0,0.0
21,9408,2979,0,0.0
25,9413,3296,0,0.0
29,9414,2227,0,0.0
33,9417,2211,0,0.0
37,9421,2106,0,0.0
41,9423,2363,0,0.0


In [159]:
def process_proposition(df):
    # Standardize column names
    df.columns = df.columns.str.replace('\n', '', regex=True)
    df.columns = df.columns.str.replace('"', '', regex=True)
    df.columns = df.columns.str.replace('.', '', regex=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.replace('_$', '', regex=True)

    # if the column name contains "bonds_-_yes", replace it with "yes"

    df.columns = df.columns.str.replace('bonds_-_yes', 'yes', regex=True)
    df.columns = df.columns.str.replace('bonds_-_no', 'no', regex=True)
    
    # Trim and rename columns to focus on necessary data
    df = df[['precinct', 'registered_voters', 'yes', 'no', 'total_votes']]
    df.columns = ['precinct', 'registered_voters', 'yes', 'no', 'votes_cast']

    # shift the precinct column up by 3 rows

    df['precinct'] = df['precinct'].shift(3)

    # Drop rows where precinct is NaN after the shift
    df = df.dropna(subset=['precinct'])
    df = df[df['precinct'].str.contains('PCT', na=False)]

    # Clean up precinct column data
    df['precinct'] = df['precinct'].str.replace('PCT ', '', regex=True).str.replace(' MB', '', regex=True)

    # Ensure numeric columns are of the correct type
    # df['registered_voters'] = pd.to_numeric(df['registered_voters'], errors='coerce').fillna(0).astype(int)
    # df['votes_cast'] = pd.to_numeric(df['votes_cast'], errors='coerce').fillna(0).astype(int)

    # Calculate turnout
    df['turnout'] = round((df['votes_cast'] / df['registered_voters']) * 100, 1).fillna(0)

    return df

In [160]:
dfA = process_proposition(dfA)

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_62288/205400271.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['precinct'] = df['precinct'].shift(3)


In [161]:
bins = [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 100]
labels = ['Less than 25%', '25-30%', '30-35%', '35-40%', '40-45%', '45-50%', '50-55%', '55-60%', '60-65%', '65-70%', '70-75%', '75% and more']

In [162]:
dfA['precinct'] = dfA['precinct'].astype(int)
dfA['yes_perc'] = (dfA['yes'] / dfA['votes_cast']) * 100
dfA['yes_perc_bin'] = pd.cut(dfA['yes_perc'], bins=bins, labels=labels, include_lowest=True)
dfA['yes_perc'] = dfA['yes_perc'].fillna('no data')
dfA['yes_perc_bin'] = dfA['yes_perc_bin'].astype(str)

In [163]:
gdf = gdf[['Prec_2022','geometry']]
gdf.Prec_2022 = gdf.Prec_2022.astype(str)

In [164]:
dfTurnout = gdf.merge(dfTurnout, right_on='precinct', left_on='Prec_2022')
dfTurnout = dfTurnout[['precinct','registered_voters','votes_cast','turnout','geometry']]
gdfTurnout = GeoDataFrame(dfTurnout, crs="EPSG:4269", geometry='geometry')

In [167]:
dfA.dtypes

precinct               int64
registered_voters    float64
yes                  float64
no                   float64
votes_cast           float64
turnout              float64
yes_perc              object
yes_perc_bin          object
dtype: object

In [169]:
# make dfA precinct string

dfA['precinct'] = dfA['precinct'].astype(str)

In [170]:
# merge 

dfA = gdf.merge(dfA, right_on='precinct', left_on='Prec_2022')

In [171]:
dfA.columns

Index(['Prec_2022', 'geometry', 'precinct', 'registered_voters', 'yes', 'no',
       'votes_cast', 'turnout', 'yes_perc', 'yes_perc_bin'],
      dtype='object')

In [172]:
# make it a geodataframe

dfA = dfA[['precinct','registered_voters','yes','no','votes_cast','turnout','yes_perc','yes_perc_bin','geometry']]
gdfA = GeoDataFrame(dfA, crs="EPSG:4269", geometry='geometry')